In [1]:
import random
from Utility import df_ops, music_ops, vmd_ops, io_ops, interpolate, cloud_ops
import pandas as pd
import os 
import pinecone
import pymeshio.common
import numpy as np

class DanceClip:
  def __init__(self, pathToVmdFolder, startFrame, numberOfFrames):
    self.pathToVmdFolder = pathToVmdFolder
    self.startFrame = startFrame
    self.numberOfFrames = numberOfFrames
  
  def __str__(self):
    return "DanceClip: " + self.pathToVmdFolder + " startFrame: " + str(self.startFrame) + " numberOfFrames: " + str(self.numberOfFrames)

def queryForSimilarPoses(pinecone, vector, songNamesToExclude):
  index = pinecone.Index(os.getenv('PINECONE_INDEX_NAME'))
  songs = index.query(
    vector=vector,
    filter={
      "animationName": { "$nin": songNamesToExclude }
    },
    top_k=1,
    include_metadata=True
  )
  matchedSongName = songs.matches[0].metadata['animationName']
  frameNumberOfMatchedSong = songs.matches[0].metadata['frameNumber']
  distance = songs.matches[0]['score']
  print("found match: " + matchedSongName + " at frame: " + str(frameNumberOfMatchedSong) + " with distance: " + str(distance))
  return matchedSongName, frameNumberOfMatchedSong

def appendFramesPreservingCenter(df, dfToAppend, buffer=0):
  if df.empty:
    lastPositionOfDf = pymeshio.common.Vector3(0, 0, 0)
  else:
    lastPositionOfDfIdx = df[df['name'] == 'センター']['frame'].idxmax()
    lastPositionOfDf = df.iloc[lastPositionOfDfIdx]['position']

  try:
    firstPositionOfDfToAppendIdx = dfToAppend[dfToAppend['name'] == 'センター']['frame'].idxmin()
    firstPositionOfDfToAppend = dfToAppend.iloc[firstPositionOfDfToAppendIdx]['position']
  except:
    if dfToAppend.empty:
      print("dfToAppend is empty")
    if len(dfToAppend[dfToAppend['name'] == 'センター']) == 0:
      print("No rows with name 'センター' in dfToAppend")
    print("Index to access:", firstPositionOfDfToAppendIdx)
    print("Length of dfToAppend:", len(dfToAppend))
    raise Exception("Error")

  differenceInPos = firstPositionOfDfToAppend - lastPositionOfDf
  differenceInPos.y = 0

  positionBoneNames = ['センター', '右足ＩＫ', '左足ＩＫ']
  dfToAppendWithAdjustedPositions = dfToAppend.copy()
  dfToAppendWithAdjustedPositions['position'] = np.where(dfToAppendWithAdjustedPositions['name'].isin(positionBoneNames), 
                                  dfToAppendWithAdjustedPositions['position'] - differenceInPos,
                                  dfToAppendWithAdjustedPositions['position'])
  return df_ops.appendFrames(df, dfToAppendWithAdjustedPositions, buffer=buffer)

def getVectorFromVectorId(pinecone, songName, framenumber):
  vectorId = songName + "-" + str(int(framenumber))
  print(vectorId)
  index = pinecone.GRPCIndex(os.getenv('PINECONE_INDEX_NAME'))
  fetchVectorId = index.fetch([vectorId])
  return fetchVectorId['vectors'][vectorId]['values']

def generateAnimationClips(numberOfFrames):
  INPUT_FOLDER = 'inputData'
  pinecone = cloud_ops.initPinecone()
  sum = 0
  end = numberOfFrames
  newSongName = random.choice([entry for entry in os.listdir(INPUT_FOLDER) if os.path.isdir(os.path.join(INPUT_FOLDER, entry))])
  print("starting with song: " + newSongName)
  pastSongs = [newSongName]
  animationClips = []
  frameNumberOfMatchedSong = None

  while sum < end:
    if sum != 0:
      print("querying for similar poses for song: " + prevSongName + " at frame: " + str(startingFrameOfNewFrames + numberOfNewFramesToAdd))
      poseVector = getVectorFromVectorId(pinecone, prevSongName, startingFrameOfNewFrames + numberOfNewFramesToAdd)
      newSongName, frameNumberOfMatchedSong = queryForSimilarPoses(pinecone, poseVector, pastSongs)
      pastSongs.append(newSongName)

    vmd = [f for f in os.listdir(INPUT_FOLDER + "\\" + newSongName) if f.endswith('.vmd')][-1]
    df = vmd_ops.getDfFromVmdFileName(INPUT_FOLDER + "\\" + newSongName + "\\" + vmd)
    numberOfNewFramesToAdd = random.randint(120, 240)
    if frameNumberOfMatchedSong and frameNumberOfMatchedSong + numberOfNewFramesToAdd > df_ops.getLastFrame(df): #handle if not enough frames left in song
      numberOfNewFramesToAdd = df_ops.getLastFrame(df) - frameNumberOfMatchedSong
    startingFrameOfNewFrames = frameNumberOfMatchedSong if sum != 0 else random.randint(0, df_ops.getLastFrame(df) - numberOfNewFramesToAdd)

    animationClips.append(DanceClip(INPUT_FOLDER + "\\" + newSongName, startingFrameOfNewFrames, numberOfNewFramesToAdd))
    sum += numberOfNewFramesToAdd
    prevSongName = newSongName
  
  return animationClips

def generateVmdAnimationFromDance(animationClips, savePath, saveName, bufferFrames=0, viewInterpolation=False):
  newAnimation = pd.DataFrame()
  newInterpolatedAnimation = pd.DataFrame()
  frameNumber = 0

  for animationClip in animationClips:
    vmd = [f for f in os.listdir(animationClip.pathToVmdFolder) if f.endswith('.vmd')][-1]
    df = vmd_ops.getDfFromVmdFileName(animationClip.pathToVmdFolder + "\\" + vmd)
    newAnimation = appendFramesPreservingCenter(newAnimation, df_ops.parseFramesFromDf(df, animationClip.startFrame, animationClip.startFrame + animationClip.numberOfFrames), bufferFrames)
    if viewInterpolation:
      interpolatedDf = df_ops.loadDfFromFeather(animationClip.pathToVmdFolder + "\\interpolatedMotion.feather")
      newInterpolatedAnimation = df_ops.appendFrames(newInterpolatedAnimation, df_ops.parseFramesFromDf(interpolatedDf, animationClip.startFrame, animationClip.startFrame + animationClip.numberOfFrames), bufferFrames)
    print("adding song: " + animationClip.pathToVmdFolder.split("\\")[-1] + "-" + str(animationClip.startFrame) + " at frame: " + str(frameNumber))
    frameNumber += animationClip.numberOfFrames + bufferFrames

  vmd_ops.saveDfToVmdFile(newAnimation, savePath + "\\" + saveName + ".vmd")
  if viewInterpolation:
    vmd_ops.saveDfToVmdFile(newInterpolatedAnimation, savePath + "\\" + saveName + "-interpolated-DEBUG.vmd")

animationClips = generateAnimationClips(1500)
generateVmdAnimationFromDance(animationClips, 'outputMotions', 'correct-center', bufferFrames=7, viewInterpolation=True)

c:\Users\daryl\AppData\Local\Programs\Python\Python311\Lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


starting with song: PV244 - Snowman
querying for similar poses for song: PV244 - Snowman at frame: 1459
PV244 - Snowman-1459
found match: PV214 - Butterfly on Your Right Shoulder -39's Giving Day Edition at frame: 2773.0 with distance: 0.992557466
querying for similar poses for song: PV214 - Butterfly on Your Right Shoulder -39's Giving Day Edition at frame: 2921.0
PV214 - Butterfly on Your Right Shoulder -39's Giving Day Edition-2921
found match: PV626 - Weekender Girl at frame: 5299.0 with distance: 0.991298795
querying for similar poses for song: PV626 - Weekender Girl at frame: 5520.0
PV626 - Weekender Girl-5520
found match: PV915 - A Single Red Leaf at frame: 6466.0 with distance: 0.983019114
querying for similar poses for song: PV915 - A Single Red Leaf at frame: 6656.0
PV915 - A Single Red Leaf-6656
found match: PV904 - Babylon at frame: 1712.0 with distance: 0.982999742
querying for similar poses for song: PV904 - Babylon at frame: 1865.0
PV904 - Babylon-1865
found match: PV311

In [2]:
for animationClip in animationClips:
  print(animationClip)

DanceClip: inputData\PV244 - Snowman startFrame: 1334 numberOfFrames: 125
DanceClip: inputData\PV214 - Butterfly on Your Right Shoulder -39's Giving Day Edition startFrame: 2773.0 numberOfFrames: 148
DanceClip: inputData\PV626 - Weekender Girl startFrame: 5299.0 numberOfFrames: 221
DanceClip: inputData\PV915 - A Single Red Leaf startFrame: 6466.0 numberOfFrames: 190
DanceClip: inputData\PV904 - Babylon startFrame: 1712.0 numberOfFrames: 153
DanceClip: inputData\PV311 - World's End Dancehall Live Mode startFrame: 5382.0 numberOfFrames: 160
DanceClip: inputData\PV211 - Iroha Uta startFrame: 1114.0 numberOfFrames: 142
DanceClip: inputData\PV901 - Strangers startFrame: 3003.0 numberOfFrames: 130
DanceClip: inputData\PV269 - 39 Music startFrame: 7110.0 numberOfFrames: 188
DanceClip: inputData\PV261 - Kimi no Taion startFrame: 3531.0 numberOfFrames: 179


In [22]:
import os
import wave
import tempfile
import shutil

def create_empty_wav(output_file):
  with wave.open(output_file, 'wb') as out_wav:
    out_wav.setparams((1, 2, 44100, 0, 'NONE', 'not compressed'))
    out_wav.writeframes(b'')

def create_empty_wav_with_params(input_file, output_file):
  with wave.open(input_file, 'rb') as wav:
    params = wav.getparams()

  with wave.open(output_file, 'wb') as out_wav:
    out_wav.setparams(params)
    out_wav.writeframes(b'')

def append_wav_files(input_file1, input_file2, output_file, start_second, end_second):
  with wave.open(input_file1, 'rb') as wav1, wave.open(input_file2, 'rb') as wav2:
    params1 = wav1.getparams()
    params2 = wav2.getparams()

    frames1 = wav1.readframes(wav1.getnframes())

    # Calculate the start and end frames for the segment in file2
    frame_rate = wav2.getframerate()
    start_frame = int(start_second * frame_rate)
    end_frame = int(end_second * frame_rate)
    n_frames = end_frame - start_frame

    # Position the file pointer and read the segment frames
    wav2.setpos(start_frame)
    frames2 = wav2.readframes(n_frames)
    print("TOTAL FRAMES: " + str(len(frames1 + frames2)) + " frameRate: " + str(frame_rate))

    with wave.open(output_file, 'wb') as out_wav:
      out_wav.setparams(params2)
      out_wav.writeframes(frames1 + frames2)

import wave
import array

def append_silence(input_file, output_file, silence_duration):
  with wave.open(input_file, 'rb') as wav:
    params = wav.getparams()
    frames = wav.readframes(wav.getnframes())

    # Create 1 second of silence
    frame_rate = params.framerate
    n_channels = params.nchannels
    sampwidth = params.sampwidth

    n_frames_silence = int(frame_rate * silence_duration)
    silence_frames = array.array("h", [0] * n_frames_silence * n_channels)

    # Convert the array to bytes
    silence_frames = silence_frames.tobytes()

    with wave.open(output_file, 'wb') as out_wav:
      out_wav.setparams(params)
      out_wav.writeframes(frames + silence_frames)

def get_wav_duration(wav_file_path):
  with wave.open(wav_file_path, 'rb') as wav_file:
    n_frames = wav_file.getnframes()
    frame_rate = wav_file.getframerate()
    duration = n_frames / float(frame_rate)
  return duration

def pullAudioFromAnimationClips(animationClips, savePath, saveName, buffer=0):
  totalFrames = 0
  newWaveName = savePath + "\\" + saveName + ".wav"
  createOutputFile = True
  for animationClip in animationClips:
    songWavName = [f for f in os.listdir(animationClip.pathToVmdFolder) if f.endswith('.wav')][-1]
    songWav = animationClip.pathToVmdFolder + "\\" + songWavName
    print(songWav)
    if createOutputFile:
      create_empty_wav_with_params(songWav, newWaveName)
      createOutputFile = False
    startSecond = animationClip.startFrame / 30
    endSecond = (animationClip.startFrame + animationClip.numberOfFrames) / 30
    append_wav_files(newWaveName, songWav, newWaveName, startSecond, endSecond)
    totalFrames += animationClip.numberOfFrames
    print("added: " + str(endSecond - startSecond) + " seconds for " + str(animationClip.numberOfFrames) + " frames. Total frames now at: " + str(totalFrames) + " frames. Total seconds now at: " + str(get_wav_duration(newWaveName)) + " seconds.")
    append_silence(newWaveName, newWaveName, buffer / 30)
    totalFrames += buffer



pullAudioFromAnimationClips(animationClips, 'outputMotions', 'audio', buffer=7)

inputData\PV244 - Snowman\pv_244_len.wav
TOTAL FRAMES: 735000 frameRate: 44100
added: 4.166666666666664 seconds for 125 frames. Total frames now at: 125 frames. Total seconds now at: 4.166666666666667 seconds.
inputData\PV214 - Butterfly on Your Right Shoulder -39's Giving Day Edition\pv_214.wav
TOTAL FRAMES: 1646400 frameRate: 44100
added: 4.933333333333323 seconds for 148 frames. Total frames now at: 280 frames. Total seconds now at: 9.333333333333334 seconds.
inputData\PV626 - Weekender Girl\pv_626.wav
TOTAL FRAMES: 2987040 frameRate: 44100
added: 7.366666666666674 seconds for 221 frames. Total frames now at: 508 frames. Total seconds now at: 16.933333333333334 seconds.
inputData\PV915 - A Single Red Leaf\pv_915.wav
TOTAL FRAMES: 4244200 frameRate: 48000
added: 6.333333333333343 seconds for 190 frames. Total frames now at: 705 frames. Total seconds now at: 22.105208333333334 seconds.
inputData\PV904 - Babylon\pv_904.wav
TOTAL FRAMES: 5268200 frameRate: 48000
added: 5.099999999999994